# Paso 3 · Embeddings + ChromaDB + Retriever (LangChain)
Genera embeddings con `all-MiniLM-L6-v2`, almacena en Chroma y consulta en lenguaje natural.

In [7]:
# 0) Limpiar entorno y clonar solo una vez
!rm -rf news-query_RPP-lab_LovatonValeria_PizarroSebastian
!git clone https://github.com/ValLovaton/news-query_RPP-lab_LovatonValeria_PizarroSebastian.git

# 1) Entrar en el repo
%cd news-query_RPP-lab_LovatonValeria_PizarroSebastian

# 2) Importar módulos necesarios (¡clave!)
import os, sys
sys.path.append(os.path.abspath("."))

# 3) Instalar dependencias
if os.path.exists("requirements.txt"):
    !pip -q install -r requirements.txt
else:
    !pip -q install feedparser pandas tiktoken

# 4) Crear carpeta de salida
os.makedirs("data/processed", exist_ok=True)

# 5) Cargar y exportar RSS
from src.rss_loader import load_rpp_rss
items = load_rpp_rss(limit=50, export_path="data/processed/rpp_latest.json")
print(f"✅ Noticias cargadas: {len(items)}")

# 6) Vista previa
import pandas as pd
df = pd.DataFrame(items)[["title", "description", "link", "date_published"]]
df.head()


Cloning into 'news-query_RPP-lab_LovatonValeria_PizarroSebastian'...
remote: Enumerating objects: 154, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 154 (delta 17), reused 7 (delta 7), pack-reused 125 (from 1)
Receiving objects: 100% (154/154), 65.13 KiB | 2.50 MiB/s, done.
Resolving deltas: 100% (71/71), done.
/content/news-query_RPP-lab_LovatonValeria_PizarroSebastian/news-query_RPP-lab_LovatonValeria_PizarroSebastian/news-query_RPP-lab_LovatonValeria_PizarroSebastian/news-query_RPP-lab_LovatonValeria_PizarroSebastian/news-query_RPP-lab_LovatonValeria_PizarroSebastian/news-query_RPP-lab_LovatonValeria_PizarroSebastian
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.

,title,description,link,date_published
0,Premier descartó reapertura del penal de El Fr...,El premier Ernesto Álvarez afirmó que es una d...,https://rpp.pe/politica/gobierno/ernesto-alvar...,"Thu, 23 Oct 2025 19:52:50 -0500"
1,Latin Billboard 2025: Karol G gana el primer p...,La cantante Karol G se llevó el galardón como ...,https://rpp.pe/musica/internacional/latin-bill...,"Thu, 23 Oct 2025 20:03:15 -0500"
2,Latin Billboard 2025: Bad Bunny consigue su pr...,¡Arriba Puerto Rico! Bad Bunny se consagró en ...,https://rpp.pe/musica/internacional/latin-bill...,"Thu, 23 Oct 2025 20:32:56 -0500"
3,LDU Quito vs. Palmeiras EN VIVO vía ESPN: se e...,Consulta todos los detalles de lo que será el ...,https://rpp.pe/futbol/copa-libertadores/ldu-qu...,"Thu, 23 Oct 2025 20:30:45 -0500"
4,Cúal fue el último temblor en México hoy 23 de...,Cuál es el ultimo temblor en México y CDMX reg...,https://rpp.pe/mundo/mexico/cual-fue-el-ultimo...,"Wed, 22 Oct 2025 06:42:35 -0500"


In [8]:
from src.embeddings_store import to_documents, upsert_chroma, build_retriever, query_retriever, results_to_df

In [9]:

# 2) Convertir a documentos y crear/actualizar Chroma
docs = to_documents(items)
vs = upsert_chroma(docs, persist_dir=".chroma", collection_name="rpp_news")
retriever = build_retriever(vs, k=8)
retriever


/content/news-query_RPP-lab_LovatonValeria_PizarroSebastian/news-query_RPP-lab_LovatonValeria_PizarroSebastian/news-query_RPP-lab_LovatonValeria_PizarroSebastian/news-query_RPP-lab_LovatonValeria_PizarroSebastian/news-query_RPP-lab_LovatonValeria_PizarroSebastian/news-query_RPP-lab_LovatonValeria_PizarroSebastian/src/embeddings_store.py:30: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  return HuggingFaceEmbeddings(model_name=EMBED_MODEL)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/content/news-query_RPP-lab_LovatonValeria_PizarroSebastian/news-query_RPP-lab_LovatonValeria_PizarroSebastian/news-query_RPP-lab_LovatonValeria_PizarroSebastian/news-query_RPP-lab_LovatonValeria_PizarroSebastian/news-query_RPP-lab_LovatonValeria_PizarroSebastian/news-query_RPP-lab_LovatonValeria_PizarroSebastian/src/embeddings_store.py:35: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vs = Chroma(collection_name=collection_name, embedding_function=embeddings, persist_directory=persist_dir)
/content/news-query_RPP-lab_LovatonValeria_PizarroSebastian/news-query_RPP-lab_LovatonValeria_PizarroSebastian/news-query_RPP-lab_LovatonValeria_PizarroSebastian/news-query_RPP-lab_LovatonValeria_PizarroSebasti

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x78feaa7d27e0>, search_kwargs={'k': 8})

In [10]:

# 3) Consulta de ejemplo
query = "Últimas noticias de economía"
results = query_retriever(retriever, query)
df = results_to_df(results)
df


/content/news-query_RPP-lab_LovatonValeria_PizarroSebastian/news-query_RPP-lab_LovatonValeria_PizarroSebastian/news-query_RPP-lab_LovatonValeria_PizarroSebastian/news-query_RPP-lab_LovatonValeria_PizarroSebastian/news-query_RPP-lab_LovatonValeria_PizarroSebastian/news-query_RPP-lab_LovatonValeria_PizarroSebastian/src/embeddings_store.py:48: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return retriever.get_relevant_documents(query)


,title,description,link,date_published
0,MEF y el presidente Jerí revisan el Presupuest...,"La ministra de Economía y Finanzas, Denisse Mi...",https://rpp.pe/economia/economia/mef-y-el-pres...,"Thu, 23 Oct 2025 19:18:25 -0500"
1,"Tras cambio en el reglamento del Congreso, arc...",Expedientes abiertos por el personal de fiscal...,https://rpp.pe/politica/elecciones/tras-cambio...,"Thu, 23 Oct 2025 19:49:41 -0500"
2,Fiscalía insiste en su pedido para que se orde...,El Ministerio Público insistió en su pedido al...,https://rpp.pe/politica/judiciales/dina-boluar...,"Thu, 23 Oct 2025 18:41:31 -0500"
3,Trump señala que México está gobernado por cár...,Trump dijo tenerle respeto a la presidenta mex...,https://rpp.pe/mundo/estados-unidos/donald-tru...,"Thu, 23 Oct 2025 17:14:05 -0500"
4,Cusco: Corpac anuncia completa operatividad en...,Corpac informó que la pista del Aeropuerto Int...,https://rpp.pe/economia/economia/cusco-corpac-...,"Thu, 23 Oct 2025 17:01:04 -0500"
5,Comisión de Energía y Minas plantea extensión ...,El predictamen de la Comisión de Energía y Min...,https://rpp.pe/politica/congreso/comision-de-e...,"Thu, 23 Oct 2025 18:59:11 -0500"
6,Villa María del Triunfo: Policía desarticula b...,"Durante un operativo policial, 14 personas fue...",https://rpp.pe/lima/policiales/villa-maria-del...,"Thu, 23 Oct 2025 17:51:04 -0500"
7,Julio Rodríguez explica los alcances de la res...,"El abogado, quien además representa al preside...",https://rpp.pe/politica/judiciales/julio-rodri...,"Thu, 23 Oct 2025 15:30:32 -0500"
